In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams["figure.figsize"] = 20 , 4

In [2]:
from AJ_ML_Library import models_classification

/Users/zhanghsiungwei/anaconda/envs/tensor/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/zhanghsiungwei/anaconda/envs/tensor/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
train = pd.read_csv("data/train_modified_3.csv")
test = pd.read_csv("data/test_modified_3.csv")

In [5]:
train.shape , test.shape

((7381, 234), (11171, 233))

In [6]:
train["fault_severity"].value_counts()

0.0    4784
1.0    1871
2.0     726
Name: fault_severity, dtype: int64

In [7]:
target = "fault_severity"
IDcol = "id"
output_colname = ["predict_0" , "predict_1" , "predict_2"]

In [8]:
train["source"] = "train"
test["source"] = "test"

In [9]:
data = pd.concat([train , test] , ignore_index=True)

In [10]:
data.shape

(18552, 235)

## Variable for Resource Type

In [12]:
model = pd.read_csv("ensemble/xgboost_models.csv")

In [14]:
model.dtypes

ModelID                         int64
Accuracy                      float64
CVScore_mean                  float64
CVScore_std                   float64
SpecifiedMetric               float64
ActualScore (manual entry)    float64
CVMethod                       object
ConfusionMatrix                object
Predictors                     object
ScoringMetric                 float64
AUC                           float64
base_score                    float64
colsample_bylevel               int64
colsample_bytree              float64
gamma                           int64
learning_rate                 float64
max_delta_step                  int64
max_depth                       int64
min_child_weight                int64
missing                         int64
n_estimators                    int64
nthread                         int64
objective                      object
reg_alpha                       int64
reg_lambda                      int64
scale_pos_weight                int64
seed        

In [16]:
print model["Feature_Importance"][1]

location               32642
feature_count          27846
location_counts        26110
feature 82             10954
feature 203             9905
feature 170             3759
feature 71              3511
feature 54              3199
feature 80              3178
feature 201             2935
feature 193             2695
feature 68              2277
event_type 11           2276
feature 232             2252
severity_type 1         2220
feature 312             2169
feature 307             1583
feature 202             1509
feature others_1        1502
feature others_0        1470
feature 73              1237
resource_type 2         1208
event_type 54           1096
feature 227             1092
feature 283             1011
feature 368              970
feature 313              943
feature 55               939
feature 291              903
event_type 35            867
feature others_2         850
feature 81               832
severity_type 2          801
feature 179              778
event_type 15 

In [17]:
data["resource_type 8and2"] = data.apply(lambda x: 1 if (x["resource_type 2"] + x["resource_type 8"]) == 2 else 0 , axis = 1)

In [18]:
data["resource_type 8and2"].value_counts()

0    17612
1      940
Name: resource_type 8and2, dtype: int64

In [19]:
def combine_8_2(x):
    s = x["resource_type 2"] + x["resource_type 8"]
    if s == 2:
        return "both"
    elif s == 0:
        return "none"
    else:
        if x["resource_type 2"] == 1:
            return "only 2"
        else:
            return "only 8"
data["resource_type 82combo"] = data.apply(combine_8_2 , axis = 1)

In [20]:
data["resource_type 82combo"].value_counts()

only 8    9328
only 2    7978
both       940
none       306
Name: resource_type 82combo, dtype: int64

In [21]:
data["resource_type 82combo"].shape

(18552,)

In [22]:
data["resource_type 82combo_lag"] = data["resource_type 82combo"].shift().fillna(-1)

In [23]:
data[["resource_type 82combo" , "resource_type 82combo_lag"]]

,resource_type 82combo,resource_type 82combo_lag
0,only 8,-1
1,only 2,only 8
2,only 2,only 2
3,only 2,only 2
4,only 2,only 2
5,only 2,only 2
6,both,only 2
7,only 2,both
8,only 2,only 2
9,only 2,only 2


In [24]:
data["resource_type_change"] = data.apply(lambda x: 1 if x["resource_type 82combo"] == x["resource_type 82combo_lag"] else 0 , axis = 1)

In [25]:
data["resource_type_change"].value_counts()

0    10286
1     8266
Name: resource_type_change, dtype: int64

In [26]:
data.drop(["resource_type 82combo" , "resource_type 82combo_lag"] , axis = 1 , inplace = True)

## Check if previous location is same

In [27]:
data["location_lag"] = data["location"].shift().fillna(-1)

In [28]:
data["location_previous_same"] = data.apply(lambda x: 1 if x["location"] == x["location_lag"] else 0 , axis = 1)

In [29]:
data.drop(["location_lag"] , axis = 1 , inplace = True)

In [31]:
data["location_previous_same"].value_counts()

0    18485
1       67
Name: location_previous_same, dtype: int64

## Divide back into test and train

In [32]:
train = data.loc[data["source"] == "train"]
test = data.loc[data["source"] == "test"]

In [33]:
train.drop("source" , axis = 1 , inplace = True)
test.drop(["source" , "fault_severity"] , axis = 1 , inplace = True)

/Users/zhanghsiungwei/anaconda/envs/tensor/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/zhanghsiungwei/anaconda/envs/tensor/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
train.to_csv("data/train_modified_4.csv" , index = False)
test.to_csv("data/test_modified_4.csv" , index = False)